<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly.express as px

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
            SELECT COUNT(*) "Количество вакансий"
            FROM public.vacancies
            '''
total_vacancies = pd.read_sql_query(query_3_1, connection)

C:\Windows\Temp\ipykernel_5576\547605699.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_vacancies = pd.read_sql_query(query_3_1, connection)


In [5]:
# результат запроса
total_vacancies

,Количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
            SELECT COUNT(*) "Количество работодателей"
            FROM public.employers
            '''
total_employers = pd.read_sql_query(query_3_2, connection)

C:\Windows\Temp\ipykernel_5576\1763035906.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_employers = pd.read_sql_query(query_3_2, connection)


In [7]:
# результат запроса
total_employers

,Количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
            SELECT COUNT(*) "Количество регионов"
            FROM public.areas
            '''
total_areas = pd.read_sql_query(query_3_3, connection)

C:\Windows\Temp\ipykernel_5576\1288392454.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_areas = pd.read_sql_query(query_3_3, connection)


In [9]:
# результат запроса
total_areas

,Количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
            SELECT COUNT(industries) "Количество сфер деятельности"
            FROM public.industries
            '''
total_industries = pd.read_sql_query(query_3_4, connection)

C:\Windows\Temp\ipykernel_5576\2825962195.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_industries = pd.read_sql_query(query_3_4, connection)


In [11]:
# результат запроса
total_industries

,Количество сфер деятельности
0,294


Из предварительного анализа данных вытекает, что:
- в базе содержится свыше 49 тысяч вакансий;
- общее количество работодателей более 23,5 тысяч, представляющие свою деятельность в 294 сферах;
- количество регионов составляет 1362 - хотя в РФ их всего 89. Сделаем допуск, что указаны регионы соседних республик, а так же - что регионами в базе считаются населенные пункты.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
            SELECT
                COUNT(v.id) cnt, 
                a.name areas
            FROM public.vacancies v
            JOIN public.areas a on v.area_id = a.id
            GROUP BY areas
            ORDER BY count(v.id) desc
            LIMIT 5
            '''
area_vacancies = pd.read_sql_query(query_4_1, connection)

C:\Windows\Temp\ipykernel_5576\1246956920.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  area_vacancies = pd.read_sql_query(query_4_1, connection)


In [13]:
# результат запроса
area_vacancies.head()

,cnt,areas
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
            SELECT COUNT(id) "Количество вакансий с зарплатой"
            FROM public.vacancies
            WHERE salary_from IS NOT NULL 
                OR salary_to IS NOT NULL 
            '''
not_null_salary = pd.read_sql_query(query_4_2, connection)

C:\Windows\Temp\ipykernel_5576\1776416032.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  not_null_salary = pd.read_sql_query(query_4_2, connection)


In [15]:
# результат запроса
not_null_salary

,Количество вакансий с зарплатой
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
            SELECT
                ROUND(AVG(salary_from)) avg_salary_from,
                ROUND(AVG(salary_to)) avg_salary_to
            FROM public.vacancies
            '''
salary_mean = pd.read_sql_query(query_4_3, connection)

C:\Windows\Temp\ipykernel_5576\1852768012.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_mean = pd.read_sql_query(query_4_3, connection)


In [17]:
# результат запроса
salary_mean

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
            SELECT
                schedule,
                employment,
                COUNT(id) cnt_vacancies
            FROM public.vacancies
            GROUP BY schedule, employment
            ORDER BY count(id) desc
            '''
schedule = pd.read_sql_query(query_4_4, connection)

C:\Windows\Temp\ipykernel_5576\1331326579.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schedule = pd.read_sql_query(query_4_4, connection)


In [19]:
# результат запроса
schedule

,schedule,employment,cnt_vacancies
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
            SELECT 
                experience, 
                COUNT(id) cnt_vacancies
            FROM public.vacancies
            GROUP BY experience
            ORDER BY count(id) asc
            '''
experience = pd.read_sql_query(query_4_5, connection)

C:\Windows\Temp\ipykernel_5576\1868450826.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience = pd.read_sql_query(query_4_5, connection)


In [21]:
# результат запроса
experience

,experience,cnt_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [22]:
# круговая диаграмма по признаку "experience"
fig = px.pie(
    experience,
    values=experience['cnt_vacancies'],
    title='Требуемый опыт работы',
    names=experience['experience']
    )

fig.update_layout(title_x=0.5, title_font_size=20)

fig.show()

Из детального анализа вакансий можно сделать следующие выводы:
    Лидерами по количеству вакансий - являются города миллионники России, Белоруссии и Казахстана.
    Почти у половины представленных вакансий не указан размер заработной платы. Скорее всего вакансии не являются узкоспециальными и не требуют большого опыта работы. 
    У подавляющего большинства представленных вакансий тип рабочего графика работы и вид трудоустройства - полный день/полная занятость, что подтверждает консервативность взглядов общества. На втором месте с большим отрывом - удаленная работа/полная занятость - говорит нам о появлении нового тренда в трудоустройстве.
    У 52% вакансий требуемый опыт работы от 1 года до 3 лет - это может говорить о том, что Работодатель не хочет тратить время на обучение и подготовку кадров, и желает минимизировать расходы в заработном фонде.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
# текст запроса
query_5_1 = f'''
            SELECT
                e.name employer_name,
                COUNT(v.id) cnt_vacancies
            FROM public.vacancies v
            JOIN public.employers e ON e.id = v.employer_id
            GROUP BY e.name
            ORDER BY COUNT(v.id) DESC
            LIMIT 5
            '''
employers = pd.read_sql_query(query_5_1, connection)

C:\Windows\Temp\ipykernel_5576\4061228731.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [24]:
# результат запроса
employers

,employer_name,cnt_vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
# текст запроса
query_5_2 = f'''
            SELECT a.name area_name,
            COUNT(e.name) cnt_employers,
            COUNT(v.id) cnt_vacancies
            FROM public.employers e
            LEFT JOIN public.vacancies v ON e.area = v.area_id
            JOIN public.areas a ON a.id  = e.area
            GROUP BY a.name
            ORDER BY cnt_vacancies ASC, cnt_employers DESC
            '''
employers_vacancies = pd.read_sql_query(query_5_2, connection)

C:\Windows\Temp\ipykernel_5576\1857881282.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [26]:
# результат запроса
employers_vacancies

,area_name,cnt_employers,cnt_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
649,Новосибирск,1149438,1149438
650,Алматы,1364132,1364132
651,Минск,2354880,2354880
652,Санкт-Петербург,6320667,6320667


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
# текст запроса
query_5_3 = f'''
            SELECT
                employer_id,
                COUNT(DISTINCT area_id) cnt_areas
            FROM public.vacancies
            GROUP BY employer_id
            ORDER BY COUNT(DISTINCT area_id) DESC
            '''
employer_regions = pd.read_sql_query(query_5_3, connection)

C:\Windows\Temp\ipykernel_5576\2282605072.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [28]:
# результат запроса
employer_regions

,employer_id,cnt_areas
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71
...,...,...
14901,810278,1
14902,810313,1
14903,810551,1
14904,810688,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
# текст запроса
query_5_4 = f'''
            SELECT COUNT(*) cnt_employers
            FROM public.employers e
            LEFT JOIN public.employers_industries i ON e.id = i.employer_id
            WHERE i.industry_id IS NULL 
            '''
industry_null = pd.read_sql_query(query_5_4, connection)

C:\Windows\Temp\ipykernel_5576\1833817398.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [30]:
# результат запроса
industry_null

,cnt_employers
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
# текст запроса
query_5_5 = f'''
            SELECT e.name employer_name
            FROM public.employers e
            LEFT JOIN public.employers_industries i ON e.id = i.employer_id
            GROUP BY e.name
            HAVING COUNT(i.industry_id) = 4
            ORDER BY e.name
            OFFSET 2 LIMIT 1
            '''
third_place = pd.read_sql_query(query_5_5, connection)

C:\Windows\Temp\ipykernel_5576\327582754.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [32]:
# результат запроса
third_place

,employer_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
# текст запроса
query_5_6 = f'''
            SELECT COUNT(*) cnt_employers
            FROM public.industries i
            LEFT JOIN public.employers_industries ei ON i.id = ei.industry_id
            WHERE i.name = 'Разработка программного обеспечения'
            '''
software_industry = pd.read_sql_query(query_5_6, connection)

C:\Windows\Temp\ipykernel_5576\1232021443.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [34]:
# результат запроса
software_industry

,cnt_employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a')
cities = list(filter(lambda city : city != '[a]', [link.text for link in links[52:68]]))
cities

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [36]:
# текст запроса
query_5_7 = f'''
(
    select a.name, count(distinct v.id) as cnt
     from VACANCIES v
     left join EMPLOYERS e on e.id = v.employer_id
     left join AREAS a on a.id = v.area_id
             where a.name in {repr(tuple(map(str, cities)))} and e.name = 'Яндекс'
     group by a.name order by 2
 )
 union all
 (
     select 'Total' as Total, sum(t.cnt) from (
      select count(distinct v.id) as cnt
             from VACANCIES v
             left join EMPLOYERS e on e.id = v.employer_id
             left join AREAS a on a.id = v.area_id
             where a.name in {repr(tuple(map(str, cities)))} and e.name = 'Яндекс'
             group by a.name
     ) as t
     
)
'''
yandex = pd.read_sql_query(query_5_7, connection)

C:\Windows\Temp\ipykernel_5576\157927891.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [37]:
# результат запроса
yandex

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

Выводы по анализу работодателей:
    Компании-лидеры по количеству вакансий представлены в сферах IT и банковские услуги.
    Крупные компании публикуют свои вакансии сразу в нескольких десятках регионов, имея разветвленную сеть.
    У трети компаний сфера деятельности не указана, а у десятой части указана "Разработка программного обеспечения".
    Крупнейшая IT-компании "Яндекс" суммарно в городах-миллионниках РФ разместила 485 вакансий - либо количество проектов резко увеличилось, либо отток персонала превысил минимальный порог допустимого и компания испытывает кадровый голод.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
# текст запроса
query_6_1 = f'''
            SELECT COUNT(*) cnt_data_vacancies
            FROM public.vacancies
            WHERE name ILIKE '%data%'
                OR name ILIKE '%данн%'
            '''
data = pd.read_sql_query(query_6_1, connection)


C:\Windows\Temp\ipykernel_5576\3595090491.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [39]:
# результат запроса
data

,cnt_data_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
# текст запроса
query_6_2 = f'''
            SELECT COUNT(*) cnt_ds_vacancies
            FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE'%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (name ILIKE '%junior%'
                OR experience = 'Нет опыта'
                OR employment = 'Стажировка')
            '''
ds = pd.read_sql_query(query_6_2, connection)


C:\Windows\Temp\ipykernel_5576\1939847567.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [41]:
# результат запроса
ds

,cnt_ds_vacancies
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [42]:
# текст запроса
query_6_3 = f'''
            SELECT COUNT(*) cnt_ds_sql_vacancies
            FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE'%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (key_skills ILIKE '%SQL%'
                OR key_skills ILIKE '%postgres%')
            '''
ds_sql = pd.read_sql_query(query_6_3, connection)

C:\Windows\Temp\ipykernel_5576\2154467091.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [43]:
# результат запроса
ds_sql

,cnt_ds_sql_vacancies
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [44]:
# текст запроса
query_6_4 = f'''
            SELECT COUNT(*) cnt_ds_py_vacancies
            FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE'%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND key_skills ILIKE '%Python%'
            '''
ds_py = pd.read_sql_query(query_6_4, connection)

C:\Windows\Temp\ipykernel_5576\2030760562.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [45]:
# результат запроса
ds_py

,cnt_ds_py_vacancies
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
# текст запроса
query_6_5 = f'''
            SELECT ROUND(AVG(1+ LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), ''))), 2) avg_ds_skills
            FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE'%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND key_skills IS NOT NULL
            '''
key_skills_ds = pd.read_sql_query(query_6_5, connection)

C:\Windows\Temp\ipykernel_5576\694841858.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [47]:
# результат запроса
key_skills_ds

,avg_ds_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
# текст запроса
query_6_6 = f'''
            SELECT experience,
                ROUND(AVG(COALESCE((salary_from + salary_to)/2, salary_to, salary_from))) avg_salary                
            FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE'%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (salary_to IS NOT NULL OR salary_from IS NOT NULL)
            GROUP BY experience
            '''
ds_salary = pd.read_sql_query(query_6_6, connection)

C:\Windows\Temp\ipykernel_5576\1614162075.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [49]:
# результат запроса
ds_salary

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

Выводы по предметному анализу вакансий, относящихся к DS:
    Отношение к сфере работы с данными имеют 1771 вакансия. Для начинающего Data Scientist есть 51 вакансия. Знаний по SQL необходимо Data Scientist в 201 вакансии. Чаще всего встречаются требования к навыкам владения Python - в 351 вакансии. Среднее количество навыков в одной вакансии для Data Scientist более 6, говорит нам о том, что для успешной работы необходимы базовые знания в разных направлениях DS.
    Средняя зарплата Data Scientist, в зависимости от опыта работы, находится в диапазоне 75-243 тыс.руб.

Дополнительно определим ТОП-10 популярных вакансий в базе HH

In [50]:
query_7 = f'''
            SELECT name vacancies_name,
                COUNT(id) cnt_vacancies
            FROM public.vacancies
            GROUP BY name
            ORDER BY cnt_vacancies DESC
            LIMIT 10
            '''
top_vacancy = pd.read_sql_query(query_7, connection)


C:\Windows\Temp\ipykernel_5576\615001825.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [51]:
top_vacancy

,vacancies_name,cnt_vacancies
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


# Общий вывод по проекту

Общий вывод по проекту:
В ходе исследования базы данных HH.ru с вакансиями можно сделать следующие выводы:
    Большинство вакансий встречается в городах-миллионниках, где находится большая часть работадателей.
    ТОП-10 популярных вакансий: все вакансии относятся к IT области. Это свидетельствует о востребованности и нехватки IT специалистов на рынке труда.
    Множество вакансий размещают работодатели, у которых указаны сферы деятельности в области IT.
    Основная масса всех вакансий это: город-миллионник, тип рабочего графика и трудоустройства - полная занятость/полный рабочий день, опыт работы - от 1 года до 3 лет.
    Примерно в половине случаев работодатель не указывает уровень заработной платы - скорее всего это средний бизнес, оптимизирующий свой фонд заработной платы.
    У начинающего специалиста по Data Science есть возможность выбора из 51 вакансии, где он может рассчитывать на зарплату чуть менее 75тыс. рублей.